## Load Packages

In [2]:
from aocd import get_data, submit
from dotenv import load_dotenv
from tqdm import tqdm

## Configure Environment

In [3]:
puzzle_day = 5
puzzle_year = 2023
load_dotenv()

True

## Load Data

In [4]:
raw_data = get_data(day=puzzle_day, year=puzzle_year)

## Solution A

### Solve

In [5]:
test_solution_a=35
test_data_a = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4"""
test_data_a = test_data_a[1:] # remove newline at beginning used to maintain formatting

### Helper Functions

In [6]:
def get_seeds(data:str):
    seed_line = data.split('\n')[0]
    seed_list = [seed for seed in seed_line.split()[1:]]
    return seed_list

In [7]:
def expand_ranges(map_ranges): 
    map_ranges = [[int(val) for val in line] for line in map_ranges]
    source = [range(line[1],line[1]+line[2],1) for line in map_ranges]
    destination = [range(line[0],line[0]+line[2],1) for line in map_ranges]
    pairings = list()
    for i,v in enumerate(source):
        pair = (v,destination[i])
        pairings.append(pair)
    return pairings

In [8]:
def get_maps(data:str):
    map_list = data.split('\n\n')[1:]
    maps = dict()
    for map_key in map_list:
        lines = map_key.split('\n')
        name = lines[0]
        keys = [line.split() for line in lines[1:]]
        pairings = expand_ranges(keys)
        maps.update({name:pairings})
    return maps


In [9]:
def get_seed_conversion_dict(seed:str,map_dict:dict):
    seed_dict = dict({'seed':seed})
    step_value = int(seed)
    for key,map_range_pairs in map_dict.items():
        step_key = key.split()[0].split('-')[-1]
        for pair in map_range_pairs:
            source,destination = pair
            if step_value in source:
                step_value = destination[source.index(step_value)]
                seed_dict.update({step_key:step_value})
                break
        seed_dict.update({step_key:step_value})
    return seed_dict

In [10]:
def solve_part_a(data:str):
    seeds = get_seeds(data)
    maps = get_maps(data)
    seed_conversions = {seed:get_seed_conversion_dict(seed,maps) for seed in seeds}
    seed_locations = {k:v.get('location') for k,v in seed_conversions.items()}
    closest_location = min(seed_locations.values())
    return int(closest_location)
assert solve_part_a(test_data_a) == test_solution_a

soln_a = solve_part_a(raw_data)

### Submit

In [11]:
if soln_a:
    submit(soln_a, part="a", day=puzzle_day, year=puzzle_year)

aocd will not submit that answer again. At 2023-12-06 14:04:20.428030-05:00 you've previously submitted 662197086 and the server responded with:
That's the right answer!  You are one gold star closer to restoring snow operations. [Continue to Part Two]


## Solution B

### Solve

In [12]:
test_solution_b=46
test_data_b = test_data_a

### Helper Functions

In [13]:
def get_seed_ranges(data:str):
    seed_line = data.split('\n')[0]
    seed_line_numbers = [int(number) for number in seed_line.split()[1:]]
    seeds = list()
    for i,v in enumerate(seed_line_numbers):
        if i % 2 == 0:
            seed_range = range(v,v+seed_line_numbers[i+1])
            seeds.append(seed_range)
    return seeds

In [14]:

import numpy as np
def subdivide_ranges(ranges,groups):
    final_ranges = list()
    for r in ranges: 
        a_r = [range(min(a),max(a)+1,1) for a in np.array_split(r,groups)]
        a_r[-1] = range(a_r[-1].start,a_r[-1].stop+1,a_r[-1].step)
        final_ranges.extend(a_r)
    return final_ranges

In [15]:
#TODO: reverse the process. Start with location (Sorted) and search back to seeds. If you get a seed match, you will 
# have found the answer. Forward search takes a loooong time! 
def solve_part_b(data:str):
    seed_ranges = get_seed_ranges(data)
    maps = get_maps(data)
    minimum_location = float('inf')
    for seed_range in seed_ranges:
        for seed in tqdm(seed_range):
            seed_conversion = get_seed_conversion_dict(seed,maps)
            minimum_location = int(seed_conversion.get('location',minimum_location)) if int(seed_conversion.get('location',minimum_location)) < minimum_location else minimum_location
    return minimum_location

assert solve_part_b(test_data_b) == test_solution_b
soln_b = solve_part_b(raw_data)

  0%|          | 0/405592018 [00:00<?, ?it/s]

 42%|████▏     | 75967194/181169206 [13:53<20:19, 86287.61it/s]  

### Submit

In [ ]:
if soln_b:
    submit(soln_b, part="b", day=puzzle_day, year=puzzle_year)